In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import pairwise_distances
from scipy.spatial import distance_matrix

#from sklearn import cluster
#from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [2]:
##Dataframe con el primer blast: Semillas vs Genomas:
df1 = pd.read_csv("/home/csar/Proyectos/Posdoc/Proyecto_pos/CsarPos/blast_EF_vs_GenomeDB/pscplos17.blast", sep = "\t", names = ['query', 'subject',
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'], engine = 'python')


##Dataframe con el segundo blast: Genomas vs Semillas: ("Revisar si en este punto los genomas corresponden
# a los de las familias expandidas")
df2 = pd.read_csv("/home/csar/Proyectos/Posdoc/Proyecto_pos/CsarPos/blast_Genomes_vs_EF/vueltalos17.blast.uniq",sep = "\t", names = ['query', 'subject',
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'], engine = 'python')


##Dataframe con el tercer blast: ExpFam_1 vs MIBiG:
df3_1 = pd.read_csv("/home/csar/Proyectos/Posdoc/Proyecto_pos/CsarPos/blast_ExpFam_vs_NP/1.fasta_ExpandedVsNp.blast",sep = "\t", names = ['query', 'subject',
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'], engine = 'python')

##Dataframe con el tercer blast: ExpFam_2 vs MIBiG:
df3_2 = pd.read_csv("/home/csar/Proyectos/Posdoc/Proyecto_pos/CsarPos/blast_ExpFam_vs_NP/2.fasta_ExpandedVsNp.blast",sep = "\t", names = ['query', 'subject',
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'], engine = 'python')


In [3]:
df1["query"][0]
## Enzimas vs Genomas:
## query: identificador | Num de fam | Función_NumSemilla | genoma

'3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu'

In [4]:
df1["subject"][0]

'gi|6666666.146852.1090|6666666.146852|NC_002935.2|D3phosphoglycerate_dehydrogenase_EC_1.1.1.95|CorynebacteriumdiphtheriaeNCTC13129s'

In [5]:
def extract_GenomeIdent(data):
    genomas = []
    genes = []
    for x in data.subject:
        x_string = x.split("|")[1]
        x_gi = x_string.split(".")[0]+"."+ x_string.split(".")[1]
        x_gen = x_string.split(".")[2]
        genomas.append(x_gi)
        genes.append(x_gen)
        
    data["genomas"] = genomas
    data["genes"] = genes
    data = pd.DataFrame.reindex(data, columns = ['query','subject', "genomas", "genes",
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'])
    return data
    
df1 = extract_GenomeIdent(df1)
df1
#df1["subject_identif"][df1["subject_identif"]=="JOEF01"]
#df1["subject_identif"].value_counts()   

,query,subject,genomas,genes,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,6666666.146852,1090,79.62,530,108,0,1,530,1,530,0.000000e+00,847.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,749927.13,1727,59.59,532,213,2,1,530,1,532,0.000000e+00,620.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,1156913.7,6305,59.20,527,214,1,1,526,1,527,0.000000e+00,613.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,6666666.104540,4265,59.20,527,214,1,1,526,1,527,0.000000e+00,612.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,446462.15,6083,60.34,532,209,2,1,530,1,532,0.000000e+00,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146852.774|6666666.146852|NC_002935...,6666666.146852,774,61.33,362,139,1,1,362,11,371,5.000000e-165,471.0
419,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.146872.438|6666666.146872|NC_017803...,6666666.146872,438,60.06,363,145,0,1,363,12,374,9.000000e-165,471.0
420,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|6666666.111572.2478|6666666.111572|NC_00948...,6666666.111572,2478,64.64,362,126,2,1,362,302,661,1.000000e-156,460.0
421,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,gi|367928.21.407|367928.21|NC_008618.1|Phospho...,367928.21,407,57.30,370,151,2,1,363,11,380,6.000000e-150,433.0


In [6]:
df1[["genomas", "genes"]].groupby(by="genomas").count()
#Podemos observar el número de copias por genoma 

,genes
genomas,
1156913.7,39
169133.3,11
290340.24,31
367928.21,15
442563.11,11
446462.15,19
452863.24,47
649831.6,27
6666666.104540,35


In [7]:
df2["query"][0]
# Genomas vs Enzimas:
# query: gi | identificadorGenoma.IdenificadorGen | IdentificadorGenoma | 


'gi|6666666.112811.3189|6666666.112811|JOEF01|Cysteine_desulfurase_EC_2.8.1.7|Allokutzneriaalbatas'

## Utilizamos la familia de enzimas (EF) como query sobre la base de datos de los genomas.

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   query          423 non-null    object 
 1   subject        423 non-null    object 
 2   genomas        423 non-null    object 
 3   genes          423 non-null    object 
 4   pc_identity    423 non-null    float64
 5   aln_length     423 non-null    int64  
 6   mismatches     423 non-null    int64  
 7   gaps_opened    423 non-null    int64  
 8   query_start    423 non-null    int64  
 9   query_end      423 non-null    int64  
 10  subject_start  423 non-null    int64  
 11  subject_end    423 non-null    int64  
 12  e_value        423 non-null    float64
 13  bitscore       423 non-null    float64
dtypes: float64(3), int64(7), object(4)
memory usage: 46.4+ KB


In [9]:
df1.head()

,query,subject,genomas,genes,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,6666666.146852,1090,79.62,530,108,0,1,530,1,530,0.0,847.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,749927.13,1727,59.59,532,213,2,1,530,1,532,0.0,620.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,1156913.7,6305,59.20,527,214,1,1,526,1,527,0.0,613.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,6666666.104540,4265,59.20,527,214,1,1,526,1,527,0.0,612.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,446462.15,6083,60.34,532,209,2,1,530,1,532,0.0,595.0


In [10]:
df1.describe()

,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
count,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,4.230000e+02,423.000000
mean,39.974917,293.378251,156.380615,4.203310,51.527187,337.513002,53.867612,341.193853,5.516379e-11,205.352246
std,12.111695,97.748995,37.400729,2.656707,45.552621,69.829679,51.258211,75.681174,9.777033e-10,181.943323
min,24.410000,92.000000,40.000000,0.000000,1.000000,279.000000,1.000000,260.000000,0.000000e+00,57.000000
25%,32.130000,238.000000,135.500000,2.000000,7.000000,304.500000,10.000000,300.000000,1.500000e-48,92.050000
50%,35.200000,265.000000,154.000000,4.000000,49.000000,313.000000,48.000000,314.000000,4.000000e-28,115.000000
75%,41.510000,324.500000,176.500000,6.000000,74.000000,325.000000,75.500000,346.000000,2.500000e-20,174.500000
max,81.600000,532.000000,280.000000,13.000000,195.000000,530.000000,302.000000,662.000000,2.000000e-08,847.000000


# Obtener las familias expandidas:

## Los genomas que aparecen en la columna "subject" son los que forman las familias expandidas. 

In [11]:
df1.head()

,query,subject,genomas,genes,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.146852.1090|6666666.146852|NC_00293...,6666666.146852,1090,79.62,530,108,0,1,530,1,530,0.0,847.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,749927.13,1727,59.59,532,213,2,1,530,1,532,0.0,620.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,1156913.7,6305,59.20,527,214,1,1,526,1,527,0.0,613.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,6666666.104540,4265,59.20,527,214,1,1,526,1,527,0.0,612.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,446462.15,6083,60.34,532,209,2,1,530,1,532,0.0,595.0


## La información descrita en un query es como sigue:
## ID | familia | función_enzima dentro de la familia | genoma

## La familia 1 tiene 4 enzimas numeradas de 1 al 4, mientras que la familia 2 consta de 3 enzimas númeradas de 1 al 3.

In [12]:
df1["query"][0]

'3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu'

## La información descrita en un subject es:
## GenInfo (gi) | Num de identificación del genoma + num de identificación del gen | Num de identificación del genoma | reference sequence | Organismo 

In [13]:
df1["subject"][0]

'gi|6666666.146852.1090|6666666.146852|NC_002935.2|D3phosphoglycerate_dehydrogenase_EC_1.1.1.95|CorynebacteriumdiphtheriaeNCTC13129s'

In [14]:
df1["query"].value_counts()

3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_1|Cglu    93
3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_2|Mtub    93
3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_3|Mtub    93
3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogenase_4|Scoe    93
3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_1|none    17
3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_2|none    17
3PGA_AMINOACIDS|2|Phosphoserine_aminotransferase_3|none    17
Name: query, dtype: int64

In [15]:
df1["subject"].value_counts()

gi|6666666.146852.1090|6666666.146852|NC_002935.2|D3phosphoglycerate_dehydrogenase_EC_1.1.1.95|CorynebacteriumdiphtheriaeNCTC13129s                         4
gi|6666666.111158.5905|6666666.111158|NC_0131311|Vancomycin_resistance_protein_VanH__Dlactate_dehydrogenase_EC_1.1.1.28|CatenulisporaacidiphilaDSM44928s    4
gi|749927.13.2117|749927.13|NC_014318.1|D3phosphoglycerate_dehydrogenase_EC_1.1.1.95|AmycolatopsismediterraneiU32s                                          4
gi|6666666.112811.5447|6666666.112811|JOEF01|D3phosphoglycerate_dehydrogenase_EC_1.1.1.95|Allokutzneriaalbatas                                              4
gi|649831.6.2685|649831.6|NC_021191.1|D3phosphoglycerate_dehydrogenase_EC_1.1.1.95|ActinoplanesspN902109s                                                   4
                                                                                                                                                           ..
gi|6666666.104540.2193|6666666.104540|AOHO01|Phospho

# Por cada enzima en cada familia vamos a obtener su correspondiente familia expandida .
## Primero vamos a identificar cada enzima según su familia y su número dentro de esa familia.


In [16]:
def extract_fam_seed(data):
    fam_seed_list = []
    for x in data["query"][:]:
        x_string = x.split("|")
        x_fam = x_string[1]
        x_enz = x_string[2].split("_")[-1]
        x_value = "F"+x_fam+"S"+x_enz
        fam_seed_list.append(x_value)
    data["(fam,seed)"] = fam_seed_list
    data = pd.DataFrame.reindex(data, columns = ['query','(fam,seed)', 'subject', "genomas", "genes",
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'])
    return data

df1 = extract_fam_seed(df1)
df1
        

,query,"(fam,seed)",subject,genomas,genes,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,F1S1,gi|6666666.146852.1090|6666666.146852|NC_00293...,6666666.146852,1090,79.62,530,108,0,1,530,1,530,0.000000e+00,847.0
1,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,F1S1,gi|749927.13.1727|749927.13|NC_014318.1|D3phos...,749927.13,1727,59.59,532,213,2,1,530,1,532,0.000000e+00,620.0
2,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,F1S1,gi|1156913.7.6305|1156913.7|NC_021252.1|D3phos...,1156913.7,6305,59.20,527,214,1,1,526,1,527,0.000000e+00,613.0
3,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,F1S1,gi|6666666.104540.4265|6666666.104540|AOHO01|D...,6666666.104540,4265,59.20,527,214,1,1,526,1,527,0.000000e+00,612.0
4,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,F1S1,gi|446462.15.6083|446462.15|NC_021252.1|D3phos...,446462.15,6083,60.34,532,209,2,1,530,1,532,0.000000e+00,595.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,F2S3,gi|6666666.146852.774|6666666.146852|NC_002935...,6666666.146852,774,61.33,362,139,1,1,362,11,371,5.000000e-165,471.0
419,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,F2S3,gi|6666666.146872.438|6666666.146872|NC_017803...,6666666.146872,438,60.06,363,145,0,1,363,12,374,9.000000e-165,471.0
420,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,F2S3,gi|6666666.111572.2478|6666666.111572|NC_00948...,6666666.111572,2478,64.64,362,126,2,1,362,302,661,1.000000e-156,460.0
421,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,F2S3,gi|367928.21.407|367928.21|NC_008618.1|Phospho...,367928.21,407,57.30,370,151,2,1,363,11,380,6.000000e-150,433.0


In [20]:
df1[["(fam,seed)", "genomas", "genes"]].groupby(by = ["(fam,seed)", "genomas"]).count()
#df1.loc[["(fam,seed)", "subject"]].groupby(by = ["(fam,seed)"]).count()

genes
(fam,seed) genomas              
F1S1       1156913.7           9
           169133.3            2
           290340.24           7
           367928.21           3
           442563.11           2
...                          ...
F2S3       6666666.146813      1
           6666666.146852      1
           6666666.146872      1
           749927.13           1
           930171.11           1

[119 rows x 1 columns]

# Extraer identificadores de genes 

In [ ]:
def gen_id(data):
    gen_id_list =[]
    for x in data["subject"][:]:
        x_string = x.split("|")
        x_id = x_string[2]
        x_name = x_string[-1]
        x_value = x_id +"|"+ x_name
        gen_id_list.append(x_value)
    data["genoma"] = gen_id_list
    data = pd.DataFrame.reindex(data, columns = ['query','(fam,seed)', 'subject', "genoma",
           'pc_identity', 'aln_length', 'mismatches', 'gaps_opened',
           'query_start', 'query_end', 'subject_start', 'subject_end',
           'e_value', 'bitscore'])
    return data
df1 = gen_id(df1)
        
df1

In [ ]:
df1[["(fam,seed)", "genoma"]].groupby(by = ["(fam,seed)"]).count()

In [ ]:
tam = len(df1.index)
dicc = {}
for i in range(tam):
    dicc[str(df1["(fam,seed)"][i])] = df1["subject"][i]
dicc

    

In [ ]:
dicc

# Obtener matrices de distancias:

In [21]:
df3_1

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,gi|749927.13.1890|749927.13|NC_014318.1|Glyoxy...,BGC0000135_BAK64657,38.34,313,175,6,16,314,20,328,9.000000e-48,166.0
1,gi|749927.13.1890|749927.13|NC_014318.1|Glyoxy...,BGC0001016_AAO62580,36.31,314,172,7,16,314,32,332,3.000000e-43,154.0
2,gi|749927.13.1890|749927.13|NC_014318.1|Glyoxy...,BGC0001017_AAF00955,35.03,314,176,8,16,314,31,331,3.000000e-41,149.0
3,gi|749927.13.1890|749927.13|NC_014318.1|Glyoxy...,BGC0001636_BBC83963,33.44,302,179,6,24,307,29,326,1.000000e-40,147.0
4,gi|749927.13.1890|749927.13|NC_014318.1|Glyoxy...,BGC0001705_ATP76237,34.71,314,177,7,16,314,31,331,2.000000e-40,146.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1726,gi|930171.11.608|930171.11|NC_015145.1|D3phosp...,BGC0000336_AAX31571,35.06,174,108,3,143,313,141,312,7.000000e-17,80.1
1727,gi|930171.11.608|930171.11|NC_015145.1|D3phosp...,BGC0000299_ERG36764,24.51,257,153,7,54,303,40,262,2.000000e-10,61.6
1728,gi|930171.11.608|930171.11|NC_015145.1|D3phosp...,BGC0001765_AMD57701,28.79,198,135,3,112,308,106,298,8.000000e-09,56.6
1729,gi|930171.11.608|930171.11|NC_015145.1|D3phosp...,BGC0001740_BBA21079,27.40,208,123,5,32,238,45,225,1.000000e-07,52.8


In [35]:
df3_1["query"].value_counts()
df3_1["subject"].value_counts()

BGC0000135_BAK64657    71
BGC0001016_AAO62580    71
BGC0001380_ALI93252    71
BGC0001765_AMD57701    71
BGC0000336_AAX31571    71
BGC0000299_ERG36764    71
BGC0000440_CAE53343    71
BGC0001178_AGS77306    71
BGC0000441_CAG15002    71
BGC0000290_AAC23583    71
BGC0001462_OLZ52462    71
BGC0000455_AEI58859    71
BGC0001461_OLZ50880    71
BGC0000419_AIG79249    71
BGC0001459_OKA09416    71
BGC0001460_EME52996    71
BGC0000317_KFH48730    71
BGC0000418_AIE77051    71
BGC0001667_AQH32479    71
BGC0000406_CAJ14043    71
BGC0001705_ATP76237    71
BGC0001636_BBC83963    71
BGC0001017_AAF00955    71
BGC0001206_AJT38686    66
BGC0001740_BBA21079    32
Name: subject, dtype: int64

In [34]:
df3_2.head()
df3_2.describe()

,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
count,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,1.800000e+01,18.000000
mean,27.960000,279.055556,174.000000,10.666667,43.555556,307.944444,28.722222,292.777778,7.031746e-06,55.255556
std,1.767634,48.957932,33.742973,2.612189,35.952947,46.907258,35.271728,48.341379,2.366881e-05,5.761763
min,23.240000,234.000000,144.000000,6.000000,20.000000,256.000000,7.000000,242.000000,2.000000e-11,44.300000
25%,27.700000,249.000000,149.250000,9.000000,23.000000,270.000000,8.250000,255.000000,3.000000e-09,51.800000
50%,28.260000,261.000000,161.000000,9.500000,32.500000,287.000000,19.000000,269.500000,6.500000e-08,54.900000
75%,28.615000,281.750000,181.750000,12.000000,44.750000,364.250000,26.000000,343.000000,5.500000e-07,59.425000
max,30.520000,372.000000,247.000000,15.000000,141.000000,375.000000,126.000000,369.000000,1.000000e-04,65.900000


In [23]:
df3_1.describe()

,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
count,1731.000000,1731.000000,1731.000000,1731.000000,1731.000000,1731.000000,1731.000000,1731.000000,1.731000e+03,1731.000000
mean,33.569965,239.569613,139.048527,5.498556,71.458694,298.403235,81.200462,311.465627,2.006521e-07,110.368804
std,8.688751,50.385579,35.145231,2.329349,46.334782,19.997031,47.449660,29.003096,3.497756e-06,69.968903
min,21.690000,73.000000,0.000000,0.000000,1.000000,200.000000,1.000000,197.000000,0.000000e+00,43.900000
25%,29.740000,220.000000,123.000000,4.000000,41.000000,288.000000,47.000000,298.000000,6.000000e-30,77.400000
50%,31.920000,244.000000,144.000000,6.000000,66.000000,299.000000,73.000000,318.000000,2.000000e-22,96.700000
75%,34.635000,275.500000,163.000000,7.000000,92.000000,313.000000,104.000000,327.000000,8.000000e-16,119.000000
max,100.000000,339.000000,213.000000,14.000000,238.000000,356.000000,231.000000,405.000000,1.000000e-04,582.000000


In [24]:
df2

,query,subject,pc_identity,aln_length,mismatches,gaps_opened,query_start,query_end,subject_start,subject_end,e_value,bitscore
0,gi|6666666.112811.3189|6666666.112811|JOEF01|C...,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,38.00,50,29,1,158,207,130,177,8.000000e-05,32.7
1,gi|6666666.111158.304|6666666.111158|NC_013131...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,32.93,82,46,3,168,243,126,204,5.000000e-06,36.6
2,gi|6666666.111572.729|6666666.111572|NC_009480...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,31.40,86,41,2,148,229,274,345,9.000000e-05,32.0
3,gi|6666666.146872.438|6666666.146872|NC_017803...,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,60.06,363,145,0,12,374,1,363,9.000000e-169,471.0
4,gi|6666666.112811.5311|6666666.112811|JOEF01|D...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,36.24,218,112,7,67,277,92,289,1.000000e-23,88.2
...,...,...,...,...,...,...,...,...,...,...,...,...
153,gi|749927.13.1890|749927.13|NC_014318.1|Glyoxy...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,40.07,282,150,5,35,306,35,307,2.000000e-49,164.0
154,gi|452863.24.129|452863.24|NC_011886.1|D3phosp...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,39.37,287,159,5,38,315,33,313,3.000000e-50,166.0
155,gi|6666666.104540.3584|6666666.104540|AOHO01|C...,3PGA_AMINOACIDS|2|Phosphoserine_aminotransfera...,33.33,90,55,4,130,218,138,223,6.000000e-05,33.1
156,gi|169133.3.1663|169133.3|LAB|Homoserine_dehyd...,3PGA_AMINOACIDS|1|Phosphoglycerate_dehydrogena...,37.78,45,28,0,342,386,442,486,3.000000e-06,37.7
